# Export/Import Fiddler Assets

Export segments and custom metrics from one Fiddler model and import them to another model, with support for:
* Cross-instance transfers (different Fiddler instances)
* Cross-model transfers (different models on same or different instances)
* Automatic schema validation with error handling for incompatible columns

## Prerequisites

* Source Fiddler instance URL and API token
* Target Fiddler instance URL and API token (can be same as source)
* Source and target models must exist
* `fiddler_utils` package installed (run cell below)

In [1]:
# Install dependencies
%pip install -q fiddler-client

# Install fiddler_utils from parent directory
# If not already installed, run from repo root: pip install -e .
import sys

sys.path.insert(0, "..")

Note: you may need to restart the kernel to use updated packages.


In [2]:
import fiddler as fdl
from fiddler_utils import (
    ConnectionManager,
    SegmentManager,
    CustomMetricManager,
    SchemaValidator,
)

print(f"Fiddler client version: {fdl.__version__}")
print("fiddler_utils: Successfully imported")

Fiddler client version: 3.10.0
fiddler_utils: Successfully imported


## Configuration

In [ ]:
# Source Fiddler Instance
SOURCE_URL = ""  # e.g., 'https://source.fiddler.ai'
SOURCE_TOKEN = ""
SOURCE_PROJECT_NAME = ""
SOURCE_MODEL_NAME = ""
SOURCE_MODEL_VERSION = ""  # Optional, leave empty for unversioned models

# Target Fiddler Instance (can be same as source)
TARGET_URL = ""  # e.g., 'https://target.fiddler.ai'
TARGET_TOKEN = ""
TARGET_PROJECT_NAME = ""
TARGET_MODEL_NAME = ""
TARGET_MODEL_VERSION = ""  # Optional, leave empty for unversioned models

# Asset Selection (empty lists = export all)
SEGMENTS_TO_EXPORT = []  # e.g., ['segment1', 'segment2'] or [] for all
CUSTOM_METRICS_TO_EXPORT = []  # e.g., ['metric1'] or [] for all


# Chart Export Options (for bonus section)
SOURCE_DASHBOARD_ID = ""  # Dashboard UUID to export charts from (leave empty to skip)
CHART_IDS_TO_EXPORT = []  # e.g., ['uuid1', 'uuid2'] or [] for none

## Setup Connections and Initialize Managers

In [4]:
# Setup connection manager for handling multiple instances
conn_mgr = ConnectionManager(log_level="WARNING")
conn_mgr.add("source", url=SOURCE_URL, token=SOURCE_TOKEN)
conn_mgr.add("target", url=TARGET_URL, token=TARGET_TOKEN)

# Initialize asset managers
segment_mgr = SegmentManager()
metric_mgr = CustomMetricManager()

print("✓ Connection manager configured")
print("✓ Asset managers initialized")

✓ Connection manager configured
✓ Asset managers initialized


## Fetch Source and Target Models

In [5]:
# Connect to source and get model
with conn_mgr.use("source"):
    source_project = fdl.Project.from_name(SOURCE_PROJECT_NAME)
    source_model_kwargs = {"project_id": source_project.id, "name": SOURCE_MODEL_NAME}
    if SOURCE_MODEL_VERSION:
        source_model_kwargs["version"] = SOURCE_MODEL_VERSION

    source_model = fdl.Model.from_name(**source_model_kwargs)
    print(f"Source model: {source_model.name} (ID: {source_model.id})")

# Connect to target and get model
with conn_mgr.use("target"):
    target_project = fdl.Project.from_name(TARGET_PROJECT_NAME)
    target_model_kwargs = {"project_id": target_project.id, "name": TARGET_MODEL_NAME}
    if TARGET_MODEL_VERSION:
        target_model_kwargs["version"] = TARGET_MODEL_VERSION

    target_model = fdl.Model.from_name(**target_model_kwargs)
    print(f"Target model: {target_model.name} (ID: {target_model.id})")

Source model: churn_classifier (ID: 8dea99c0-6724-46df-a2de-5d542ed7f272)
Target model: bank_churn_simple_monitoring_v2 (ID: 31f97c25-fd0b-44ad-9860-9077747a0119)


## Schema Comparison

Compare source and target model schemas to identify potential compatibility issues.

In [6]:
# Get schema information from both models
with conn_mgr.use("source"):
    source_columns = SchemaValidator.get_model_columns(source_model)
    print(f"Source model has {len(source_columns)} columns")

with conn_mgr.use("target"):
    target_columns = SchemaValidator.get_model_columns(target_model)
    print(f"Target model has {len(target_columns)} columns")

with conn_mgr.use("source"):
    comparison = SchemaValidator.compare_schemas(source_model, target_model)

print("\n" + "=" * 60)
print("SCHEMA COMPARISON")
print("=" * 60)
print(f"\nCommon columns: {len(comparison.in_both)}")

if comparison.only_in_source:
    print(
        f"\n⚠️  Columns in SOURCE but MISSING in TARGET ({len(comparison.only_in_source)}):"
    )
    for col in sorted(comparison.only_in_source):
        col_info = source_columns[col]
        print(f"  - {col} ({col_info.role}, {col_info.data_type})")
else:
    print("\n✅ No missing columns - all source columns exist in target")

if comparison.only_in_target:
    print(
        f"\n📋 Columns in TARGET but NOT in SOURCE ({len(comparison.only_in_target)}):"
    )
    for col in sorted(list(comparison.only_in_target)[:5]):
        col_info = target_columns[col]
        print(f"  - {col} ({col_info.role}, {col_info.data_type})")
    if len(comparison.only_in_target) > 5:
        print(f"  ... and {len(comparison.only_in_target) - 5} more")

if comparison.type_mismatches:
    print(f"\n⚠️  Data type DIFFERENCES ({len(comparison.type_mismatches)}):")
    for col, (source_type, target_type) in list(comparison.type_mismatches.items())[:5]:
        print(f"  - {col}: source={source_type} → target={target_type}")
else:
    print("\n✅ All common columns have matching data types")

print(
    f"\n{'✅' if comparison.is_compatible else '⚠️ '} Schema compatibility: {comparison.is_compatible}"
)
print("=" * 60)

Source model has 13 columns
Target model has 14 columns

SCHEMA COMPARISON

Common columns: 11

⚠️  Columns in SOURCE but MISSING in TARGET (2):
  - decisions (ColumnRole.DECISION, None)
  - probability_churn (ColumnRole.OUTPUT, None)

📋 Columns in TARGET but NOT in SOURCE (3):
  - customer_id (ColumnRole.METADATA, None)
  - predicted_churn (ColumnRole.OUTPUT, None)
  - timestamp (ColumnRole.METADATA, None)

✅ All common columns have matching data types

⚠️  Schema compatibility: False


In [7]:
print("\n" + "=" * 60)
print("EXPORTING SEGMENTS")
print("=" * 60)

with conn_mgr.use("source"):
    # Export segments (filtered by name if specified)
    exported_segments = segment_mgr.export_assets(
        model_id=source_model.id,
        names=SEGMENTS_TO_EXPORT if SEGMENTS_TO_EXPORT else None,
    )

    print(f"\n✓ Exported {len(exported_segments)} segment(s)")

    for seg_data in exported_segments:
        print(f"\n  Segment: {seg_data.name}")
        print(f"    Definition: {seg_data.data['definition']}")
        print(f"    Referenced columns: {seg_data.referenced_columns}")


EXPORTING SEGMENTS

✓ Exported 15 segment(s)

  Segment: California Customers
    Definition: geography=='California'
    Referenced columns: set()

  Segment: California Customers between 30 and 60
    Definition: (age<60 or age>30) and geography=='California'
    Referenced columns: set()

  Segment: California Customers over 60
    Definition: age>60 and geography=='California'
    Referenced columns: set()

  Segment: California Customers under 30
    Definition: age<30 and geography=='California'
    Referenced columns: set()

  Segment: Female Customers
    Definition: gender=='Female'
    Referenced columns: set()

  Segment: Florida Customers
    Definition: geography=='Florida'
    Referenced columns: set()

  Segment: Has CC
    Definition: hascrcard==1
    Referenced columns: set()

  Segment: Has no CC
    Definition: hascrcard==0
    Referenced columns: set()

  Segment: Hawaii Customers
    Definition: geography=='Hawaii'
    Referenced columns: set()

  Segment: Male Cu

## Import Segments

Import segments to target model with automatic validation.

In [8]:
print("\n" + "=" * 60)
print("IMPORTING SEGMENTS")
print("=" * 60)

with conn_mgr.use("target"):
    # Import with validation and error handling
    segment_result = segment_mgr.import_assets(
        target_model_id=target_model.id,
        assets=exported_segments,
        validate=True,
        dry_run=False,
        skip_invalid=True,
        overwrite=False,
    )

    print("\nResults:")
    print(f"  ✅ Successfully imported: {segment_result.successful}")
    print(f"  🔄 Skipped (existing): {segment_result.skipped_existing}")
    print(f"  ⊘  Skipped (invalid): {segment_result.skipped_invalid}")
    print(f"  ❌ Failed: {segment_result.failed}")

    if segment_result.errors:
        print("\n  Errors:")
        for name, error in segment_result.errors:
            print(f"    - {name}: {error}")


IMPORTING SEGMENTS

Results:
  ✅ Successfully imported: 0
  🔄 Skipped (existing): 15
  ⊘  Skipped (invalid): 0
  ❌ Failed: 0


## Export Custom Metrics

Export custom metrics from source model using `CustomMetricManager`.

In [9]:
print("\n" + "=" * 60)
print("EXPORTING CUSTOM METRICS")
print("=" * 60)

with conn_mgr.use("source"):
    # Export custom metrics (filtered by name if specified)
    exported_metrics = metric_mgr.export_assets(
        model_id=source_model.id,
        names=CUSTOM_METRICS_TO_EXPORT if CUSTOM_METRICS_TO_EXPORT else None,
    )

    print(f"\n✓ Exported {len(exported_metrics)} custom metric(s)")

    for metric_data in exported_metrics:
        print(f"\n  Metric: {metric_data.name}")
        print(f"    Definition: {metric_data.data['definition']}")
        print(f"    Referenced columns: {metric_data.referenced_columns}")

        # Show complexity info
        metadata = metric_data.data["metadata"]
        metric_type = "Aggregation" if metadata["is_aggregation"] else "Simple"
        print(f"    Type: {metric_type}")
        if metadata["functions_used"]:
            print(f"    Functions: {', '.join(metadata['functions_used'])}")


EXPORTING CUSTOM METRICS

✓ Exported 9 custom metric(s)

  Metric: Disparate Impact Female against Male
    Definition: (sum(if(("probability_churn">0.8 and "gender"== 'Female'), 1, 0))/sum(if(("gender"== 'Female'), 1, 0)))/(sum(if(("probability_churn">0.8 and "gender"== 'Male'), 1, 0))/sum(if(("gender"== 'Male'), 1, 0)))
    Referenced columns: {'probability_churn', 'gender'}
    Type: Aggregation
    Functions: sum, if

  Metric: Disparate Impact Non Binary against Male
    Definition: (sum(if(("probability_churn">0.8 and "gender"== 'Nonbinary'), 1, 0))/sum(if(("gender"== 'Nonbinary'), 1, 0)))/(sum(if(("probability_churn">0.8 and "gender"== 'Male'), 1, 0))/sum(if(("gender"== 'Male'), 1, 0)))
    Referenced columns: {'probability_churn', 'gender'}
    Type: Aggregation
    Functions: sum, if

  Metric: Equal Opportunity True Positive Rate
    Definition: sum(if(tp(), 1, 0)) / sum(if(tp(), 1, 0)+if(fn(), 1, 0))
    Referenced columns: set()
    Type: Aggregation
    Functions: sum, if

## Import Custom Metrics

Import custom metrics to target model with automatic validation.

In [ ]:
print("\n" + "=" * 60)
print("IMPORTING CUSTOM METRICS")
print("=" * 60)

with conn_mgr.use("target"):
    # Import with validation and error handling
    metric_result = metric_mgr.import_assets(
        target_model_id=target_model.id,
        assets=exported_metrics,
        validate=True,
        dry_run=False,
        skip_invalid=True,
        overwrite=False,
    )

    print("\nResults:")
    print(f"  ✅ Successfully imported: {metric_result.successful}")
    print(f"  🔄 Skipped (existing): {metric_result.skipped_existing}")
    print(f"  ⊘  Skipped (invalid): {metric_result.skipped_invalid}")
    print(f"  ❌ Failed: {metric_result.failed}")

    if metric_result.errors:
        print("\n  Errors:")
        for name, error in metric_result.errors:
            print(f"    - {name}: {error}")


IMPORTING CUSTOM METRICS

Results:
  ✅ Successfully imported: 0
  🔄 Skipped (existing): 7
  ⊘  Skipped (invalid): 2
  ❌ Failed: 0

  Errors:
    - Disparate Impact Female against Male: Missing columns: ['probability_churn']
    - Disparate Impact Non Binary against Male: Missing columns: ['probability_churn']


## Summary Report

In [ ]:
print("\n" + "=" * 60)
print("EXPORT/IMPORT SUMMARY")
print("=" * 60)

print("\nSegments:")
print(f"  Total exported: {len(exported_segments)}")
print(f"  Successfully imported: {segment_result.successful}")
print(f"  Skipped (existing): {segment_result.skipped_existing}")
print(f"  Skipped (invalid): {segment_result.skipped_invalid}")
print(f"  Failed: {segment_result.failed}")

print("\nCustom Metrics:")
print(f"  Total exported: {len(exported_metrics)}")
print(f"  Successfully imported: {metric_result.successful}")
print(f"  Skipped (existing): {metric_result.skipped_existing}")
print(f"  Skipped (invalid): {metric_result.skipped_invalid}")
print(f"  Failed: {metric_result.failed}")

total_success = segment_result.successful + metric_result.successful
total_skipped_existing = (
    segment_result.skipped_existing + metric_result.skipped_existing
)
total_skipped_invalid = segment_result.skipped_invalid + metric_result.skipped_invalid
total_failed = segment_result.failed + metric_result.failed
total_exported = len(exported_segments) + len(exported_metrics)

print("\n" + "=" * 60)
print(f"OVERALL: {total_success}/{total_exported} assets successfully imported")
if total_skipped_existing > 0:
    print(f"  {total_skipped_existing} skipped (already exist on target)")
if total_skipped_invalid > 0:
    print(f"  {total_skipped_invalid} skipped (validation errors)")
if total_failed > 0:
    print(f"  {total_failed} failed during import")
print("=" * 60)


EXPORT/IMPORT SUMMARY

Segments:
  Total exported: 15
  Successfully imported: 0
  Skipped (existing): 15
  Skipped (invalid): 0
  Failed: 0

Custom Metrics:
  Total exported: 9
  Successfully imported: 0
  Skipped (existing): 7
  Skipped (invalid): 2
  Failed: 0

OVERALL: 0/24 assets successfully imported
  22 skipped (already exist on target)
  2 skipped (validation errors)


## Export/Import Charts

Demonstrate cross-instance chart transfer using `ChartManager`.

This section shows how fiddler_utils simplifies chart transfers.

**To use this section:**
1. Set `SOURCE_DASHBOARD_ID` in the configuration cell above (find dashboard ID in Fiddler UI URL)
2. OR set `CHART_IDS_TO_EXPORT` to manually specify chart UUIDs
3. Run the cells below to export and import charts

**Note:** Chart API uses unofficial Fiddler endpoints and may change without notice.

In [12]:
from fiddler_utils import ChartManager

print("\n" + "=" * 60)
print("INITIALIZING CHART MANAGERS")
print("=" * 60)

# Create separate managers for source and target
# Each needs its own URL/token for RequestClient
source_chart_mgr = ChartManager(url=SOURCE_URL, token=SOURCE_TOKEN)
target_chart_mgr = ChartManager(url=TARGET_URL, token=TARGET_TOKEN)

print("\n✓ Source ChartManager initialized")
print("✓ Target ChartManager initialized")


INITIALIZING CHART MANAGERS

✓ Source ChartManager initialized
✓ Target ChartManager initialized


In [ ]:
print("\n" + "=" * 60)
print("EXPORTING CHARTS")
print("=" * 60)

# Determine export method
if SOURCE_DASHBOARD_ID:
    print(f"\nExporting charts from dashboard: {SOURCE_DASHBOARD_ID}")
elif CHART_IDS_TO_EXPORT:
    print(f"\nExporting {len(CHART_IDS_TO_EXPORT)} charts by ID")
else:
    print("\n⚠️  No SOURCE_DASHBOARD_ID or CHART_IDS_TO_EXPORT specified.")
    print("   Set one of these in the configuration cell to export charts.")
    exported_charts = []

if SOURCE_DASHBOARD_ID or CHART_IDS_TO_EXPORT:
    with conn_mgr.use("source"):
        try:
            # Export charts using dashboard_id or chart_ids
            exported_charts = source_chart_mgr.export_charts(
                dashboard_id=SOURCE_DASHBOARD_ID if SOURCE_DASHBOARD_ID else None,
                chart_ids=CHART_IDS_TO_EXPORT if CHART_IDS_TO_EXPORT else None,
            )

            print(f"\n✓ Exported {len(exported_charts)} chart(s)\n")

            # Display exported chart details
            for i, chart in enumerate(exported_charts, 1):
                print(f"{i}. {chart.get('title', 'Untitled')}")
                print(f"   Type: {chart.get('query_type', 'unknown')}")

                # Show data source info
                data_source = chart.get("data_source", {})
                queries = data_source.get("queries", [])

                if queries:
                    # Show first query details for monitoring charts
                    query = queries[0]
                    metric_info = []

                    if "baseline_name" in query:
                        metric_info.append(f"baseline={query['baseline_name']}")
                    if query.get("metric_type") == "custom":
                        metric_info.append(
                            f"custom_metric={query.get('metric', 'N/A')}"
                        )
                    if "segment" in query and query["segment"]:
                        metric_info.append(f"segment={query['segment']}")

                    if metric_info:
                        print(f"   Dependencies: {', '.join(metric_info)}")
                print()
        except Exception as e:
            print(f"\n❌ Failed to export charts: {e}")
            exported_charts = []


EXPORTING CHARTS

Exporting charts from dashboard: dbd97cc9-3cfb-4d22-8159-a025ced9b455

✓ Exported 12 chart(s)

1. Model Performance - Monthly
   Type: ANALYTICS

2. Revenue Impact By State
   Type: MONITORING
   Dependencies: custom_metric=3662284e-df05-4874-8904-3af32261e98e, segment={'id': 'd06c74a4-7480-46c5-a10f-659e79527ffe'}

3. Prediction Drift
   Type: MONITORING

4. Accuracy By State
   Type: MONITORING
   Dependencies: segment={'id': 'd06c74a4-7480-46c5-a10f-659e79527ffe'}

5. Decision Distribution 
   Type: ANALYTICS

6. Decision Volume Tracking
   Type: MONITORING

7. Churn Probability - Gender
   Type: MONITORING
   Dependencies: segment={'id': '70298256-e1b5-49c1-abd4-3bc1908ecf5d'}

8. Churn Probability - Geography
   Type: MONITORING
   Dependencies: segment={'id': 'ed229f55-370e-48af-acef-93176fcb5896'}

9. Group Benefit - Gender
   Type: MONITORING
   Dependencies: custom_metric=b507cad8-c257-4467-b3a3-96dfc0f4e67f, segment={'id': '70298256-e1b5-49c1-abd4-3bc1908ec

In [ ]:
if not exported_charts:
    print("\n⊘ No charts to analyze. Skipping dependency analysis.")
else:
    print("=" * 60)
    print("ANALYZING CHART DEPENDENCIES")
    print("=" * 60)

    # Collect all dependencies from charts
    baseline_refs = set()
    metric_refs = set()
    segment_refs = set()

    for chart in exported_charts:
        data_source = chart.get("data_source", {})
        queries = data_source.get("queries", [])

        for query in queries:
            if "baseline_name" in query:
                baseline_refs.add(query["baseline_name"])
            if query.get("metric_type") == "custom" and "metric" in query:
                metric_refs.add(query["metric"])
            if "segment" in query and query["segment"]:
                segment_val = query["segment"]
                if isinstance(segment_val, dict):
                    # May have segment ID or name
                    if "name" in segment_val:
                        segment_refs.add(segment_val["name"])
                elif isinstance(segment_val, str):
                    segment_refs.add(segment_val)

    print(f"\nCharts reference:")
    print(
        f"  {len(baseline_refs)} baseline(s): {sorted(baseline_refs) if baseline_refs else 'None'}"
    )
    print(
        f"  {len(metric_refs)} custom metric(s): {sorted(list(metric_refs)[:3]) if metric_refs else 'None'}"
    )
    print(
        f"  {len(segment_refs)} segment(s): {sorted(list(segment_refs)[:3]) if segment_refs else 'None'}"
    )

    print("\n💡 These dependencies must exist on target for successful import")

ANALYZING CHART DEPENDENCIES

Charts reference:
  0 baseline(s): None
  2 custom metric(s): ['3662284e-df05-4874-8904-3af32261e98e', 'b507cad8-c257-4467-b3a3-96dfc0f4e67f']
  0 segment(s): None

💡 These dependencies must exist on target for successful import


In [ ]:
if not exported_charts:
    print("\n⊘ No charts to validate. Skipping dry run.")
else:
    print("\n" + "=" * 60)
    print("DRY RUN: VALIDATE CHART IMPORT")
    print("=" * 60)

    with conn_mgr.use("target"):
        # Perform dry run to validate without creating charts
        dry_result = target_chart_mgr.import_charts(
            target_project_id=target_project.id,
            target_model_id=target_model.id,
            charts=exported_charts,
            validate=True,
            dry_run=True,  # Preview only
        )

        print(f"\nDry run results:")
        print(f"  ✅ Would succeed: {dry_result['successful']}")
        print(f"  ❌ Would fail: {dry_result['failed']}")

        if dry_result.get("errors"):
            print(f"\n⚠️  Potential errors ({len(dry_result['errors'])}):")
            for title, error in dry_result["errors"][:5]:  # Show first 5
                print(f"    • {title}")
                print(f"      {error}")
            if len(dry_result["errors"]) > 5:
                print(f"    ... and {len(dry_result['errors']) - 5} more")
        else:
            print("\n✅ All charts validated successfully")


DRY RUN: VALIDATE CHART IMPORT

Dry run results:
  ✅ Would succeed: 12
  ❌ Would fail: 0

✅ All charts validated successfully


In [ ]:
if not exported_charts:
    print("\n⊘ No charts to import. Skipping import.")
    chart_result = {"successful": 0, "failed": 0, "errors": []}
else:
    print("\n" + "=" * 60)
    print("IMPORTING CHARTS")
    print("=" * 60)

    with conn_mgr.use("target"):
        # Perform actual import
        chart_result = target_chart_mgr.import_charts(
            target_project_id=target_project.id,
            target_model_id=target_model.id,
            charts=exported_charts,
            validate=True,
            dry_run=False,  # Actually create charts
        )

        print("\nResults:")
        print(f"  ✅ Successfully imported: {chart_result['successful']}")
        print(f"  ❌ Failed: {chart_result['failed']}")

        if chart_result.get("errors"):
            print(f"\n  Errors encountered:")
            for title, error in chart_result["errors"]:
                print(f"    • {title}")
                print(f"      {error}")

        # Update overall summary
        if chart_result["successful"] > 0:
            print(
                f"\n✓ Successfully imported {chart_result['successful']} chart(s) to target model"
            )


IMPORTING CHARTS

Results:
  ✅ Successfully imported: 12
  ❌ Failed: 0

✓ Successfully imported 12 chart(s) to target model
